大家好！今天我来教大家通过新浪微博api抓微博评论，并生成评论关系图，看看哪个键盘侠嘴最臭！

# 数据抓取

In [34]:
#抓取的包
import urllib
import urllib.request

#json文件解析
import json

#数据处理包
import pandas as pd
import numpy as np

首先，各位在用微博api之前，需要申请一下成为微博开发者：[微博开放平台](https://open.weibo.com/)

然后，新建一个应用，随便什么应用都可以，应用不需要审核，只需要创建就行

今天我们要用的api接口是这个：[根据微博ID返回某条微博的评论列表](https://open.weibo.com/wiki/2/comments/show)

然后打开:[api测试工具](http://open.weibo.com/tools/console?uri=comments/show&amp;httpmethod=GET&amp;key1=id)

这个api的请求参数是：
<img src="https://gitee.com/ni1o1/pygeo-tutorial/raw/master/resource/weibo1.png"  style="width:600px">

在api测试工具里面，我们可以找到我们的access_token！那么，把这些参数放到api测试工具里面增加api参数，点击调用参口，可以看到上面的对话框里面出现了我们要的链接
<img src="https://gitee.com/ni1o1/pygeo-tutorial/raw/master/resource/weibo2.png"  style="width:600px">

先来抓一条微博，是讲女权的，微博的id是4503823458057845，里面的评论可精彩了值得一抓！[戳这里看内容](https://weibo.com/1887344341/J1BsRxOdf?from=page_1006061887344341_profile&wvr=6&mod=weibotime)

In [78]:
#请求URL
url = r'https://api.weibo.com/2/comments/show.json'
#请求参数：
para = r'id=4503823458057845&access_token=xxxxxxxxxxxxxxxxxxxxxxxxx&count=200&page=1'

#构建url
url = url+'?'+para
#抓取
request = urllib.request.Request(url)
response = urllib.request.urlopen(request)
data = response.read().decode('utf-8')
#读取json
data = json.loads(data)

In [ ]:
#那么数据就已经被我们抓下来了，可以看看长啥样
data

In [79]:
#这个json文件里面的comments可以直接转换为pd的dataframe
data = pd.DataFrame(data['comments'])
data.head(5)

,created_at,id,rootid,rootidstr,floor_number,text,disable_reply,user,mid,idstr,status,reply_comment,reply_original_text,readtimetype,comment_badge,extend_info
0,Tue May 12 22:50:15 +0800 2020,4503892094632524,4503877749088959,4503877749088959,0,回复@青荼蘼:就你这阴阳怪气的评论，普通人看到只会厌恶，这对女权的推广是极为不利的，戾气太重...,0,"{'id': 3882791691, 'idstr': '3882791691', 'cla...",4503892094632524,4503892094632524,"{'visible': {'type': 0, 'list_id': 0}, 'create...",{'created_at': 'Tue May 12 21:53:15 +0800 2020...,就你这阴阳怪气的评论，普通人看到只会厌恶，这对女权的推广是极为不利的，戾气太重，导致很多对女...,comment,NaN,NaN
1,Tue May 12 22:50:01 +0800 2020,4503892030895239,4503842027558210,4503842027558210,0,回复@喷火的龙之传人:反婚反育不就好了。女性基本找不到同意让冠姓权的男性，楼里那位女性反婚反...,0,"{'id': 7041459439, 'idstr': '7041459439', 'cla...",4503892030895239,4503892030895239,"{'visible': {'type': 0, 'list_id': 0}, 'create...",{'created_at': 'Tue May 12 19:31:19 +0800 2020...,反婚反育不就好了。女性基本找不到同意让冠姓权的男性，楼里那位女性反婚反育应该影响不到你吧。。,comment,NaN,NaN
2,Tue May 12 22:49:14 +0800 2020,4503891834205300,4503825996628106,4503825996628106,0,回复@Digital曦:你不是，那人家也不是骂你。作为一个网民随意发表点言论，引来一群蛆骂，...,0,"{'id': 1167831071, 'idstr': '1167831071', 'cla...",4503891834205300,4503891834205300,"{'visible': {'type': 0, 'list_id': 0}, 'create...",{'created_at': 'Tue May 12 22:46:13 +0800 2020...,你不是，那人家也不是骂你。作为一个网民随意发表点言论，引来一群蛆骂，那人家当然要骂回去。,comment,NaN,NaN
3,Tue May 12 22:48:57 +0800 2020,4503891762916223,4503823655872375,4503823655872375,0,回复@鹧鸪孤鶱:可她就是贱驴啊，让孩子随父姓不是驴是什么。,0,"{'id': 5506099264, 'idstr': '5506099264', 'cla...",4503891762916223,4503891762916223,"{'visible': {'type': 0, 'list_id': 0}, 'create...",{'created_at': 'Tue May 12 18:18:19 +0800 2020...,可她就是贱驴啊，让孩子随父姓不是驴是什么。,comment,NaN,NaN
4,Tue May 12 22:48:52 +0800 2020,4503891741431696,4503824926864646,4503824926864646,0,回复@l若愚l:你家过日子不是互相伤害，你家是你和你爸俩大鼎占你妈便宜……你能把对立理解成互...,0,"{'id': 1549170040, 'idstr': '1549170040', 'cla...",4503891741431696,4503891741431696,"{'visible': {'type': 0, 'list_id': 0}, 'create...",{'created_at': 'Tue May 12 22:39:34 +0800 2020...,你家过日子不是互相伤害，你家是你和你爸俩大鼎占你妈便宜……你能把对立理解成互相伤害真是智商低...,comment,NaN,NaN


不过，这里咱把全部评论抓下来，怎么抓呢？  
可以看到，上面请求参数里面有个page参数  

没错！很容易！就是page从1到正无穷，直到抓到评论数量为0就停

In [ ]:
#初始化datalen参数记录每次抓到的评论数
datalen = 1
#从第一页开始抓
page = 1
#初始化url
url = r'https://api.weibo.com/2/comments/show.json'
#初始化请求参数
para = r'id=4503823458057845&access_token=xxxxxxxxxxxxxxxxxxxxxxxxx&count=200&page=1'
#每次抓的数据存储到一个list中
result = []

while datalen>0:
    #请求参数：
    para1 = para[:-1]+str(page)
    #构建url
    url1 = url+'?'+para1
    print(url1)
    
    #抓取
    request = urllib.request.Request(url1)
    response = urllib.request.urlopen(request)
    data = response.read().decode('utf-8')
    
    #读取json
    data = json.loads(data)

    #这个json文件里面的comments可以直接转换为pd的dataframe
    data = pd.DataFrame(data['comments'])
    
    #看看抓到多少条数据
    datalen = len(data)
    
    #如果有数据，就放进result里
    if datalen>0:
        result.append(data)
    #抓下一页
    page+=1

#合并为一个DataFrame
result = pd.concat(result)

In [83]:
len(result)

829

好的我一共抓到了829条微博评论

# 数据处理

In [84]:
result.head(5)

,comment_badge,created_at,disable_reply,extend_info,floor_number,id,idstr,mid,readtimetype,reply_comment,reply_original_text,rootid,rootidstr,status,text,user
0,[{'pic_url': 'https://h5.sinaimg.cn/upload/100...,Tue May 12 23:01:58 +0800 2020,0,NaN,347,4503895039318273,4503895039318273,4503895039318273,comment,NaN,NaN,4503895039318273,4503895039318273,"{'visible': {'type': 0, 'list_id': 0}, 'create...",呼吁大家孩子出生后称呼18位身份证号，直到孩子成年后让ta决定自己的姓名姓氏[爱你]~关注孩...,"{'id': 1496160994, 'idstr': '1496160994', 'cla..."
1,NaN,Tue May 12 23:01:31 +0800 2020,0,"{'danmaku_time': '3638', 'danmaku_from': '0'}",346,4503894929895229,4503894929895229,4503894929895229,comment,NaN,NaN,4503894929895229,4503894929895229,"{'visible': {'type': 0, 'list_id': 0}, 'create...",咦？平时有男人嘴炮中国女生是母狗、是eg、是性资源的时候，怎么没见这么多男的义愤填膺？现在倒...,"{'id': 5589656207, 'idstr': '5589656207', 'cla..."
2,NaN,Tue May 12 23:01:01 +0800 2020,0,NaN,0,4503894799684876,4503894799684876,4503894799684876,comment,{'created_at': 'Tue May 12 20:28:14 +0800 2020...,那没事了[允悲][允悲]可能是我没见到吧，我的,4503827548019096,4503827548019096,"{'visible': {'type': 0, 'list_id': 0}, 'create...",回复@一块废柄:那没事了[允悲][允悲]可能是我没见到吧，我的,"{'id': 5690080210, 'idstr': '5690080210', 'cla..."
3,NaN,Tue May 12 22:57:36 +0800 2020,0,NaN,0,4503893939440479,4503893939440479,4503893939440479,comment,{'created_at': 'Tue May 12 22:50:53 +0800 2020...,谁强迫她了？恩和开始说的也只是陈述句“孩子随父姓”，现在整的我以为有拿枪逼着她给娃改姓呢。不...,4503877749088959,4503877749088959,"{'visible': {'type': 0, 'list_id': 0}, 'create...",回复@我家狗子叫小猫:谁强迫她了？恩和开始说的也只是陈述句“孩子随父姓”，现在整的我以为有拿...,"{'id': 2274183742, 'idstr': '2274183742', 'cla..."
4,NaN,Tue May 12 22:57:26 +0800 2020,0,NaN,0,4503893901477855,4503893901477855,4503893901477855,comment,{'created_at': 'Tue May 12 19:31:19 +0800 2020...,张小美那青春又美丽的脸被我颜射了？我自己也彻底懵逼了。而接下来我要面对的是张小美的勃然大怒—...,4503842027558210,4503842027558210,"{'visible': {'type': 0, 'list_id': 0}, 'create...",回复@喷火的龙之传人:张小美那青春又美丽的脸被我颜射了？我自己也彻底懵逼了。而接下来我要面对...,"{'id': 1711692274, 'idstr': '1711692274', 'cla..."


下一步我们来看看是谁在评论回复谁  
首先获取每条评论是谁发的，用户的的信息存储在user列中

In [97]:
#发评论的人
result['username'] = pd.DataFrame(list(result['user']))['name']

In [98]:
#有些评论是回复别人的，回复的信息存储在reply_comment列中。
#有些有有些没，我们把这些有回复的提取出来
pd.DataFrame(list(result[-result['reply_comment'].isnull()]['reply_comment'])).head(5)

,created_at,id,rootid,rootidstr,floor_number,text,disable_reply,user,mid,idstr,extend_info,comment_badge
0,Tue May 12 20:28:14 +0800 2020,4503856354178157,4503827548019096,4503827548019096,0,回复@ZWWzwc:我身边搞女权的都是985211清华北大伦敦大学啊[疑问]有什么问题吗,0,"{'id': 7274821563, 'idstr': '7274821563', 'cla...",4503856354178157,4503856354178157,NaN,NaN
1,Tue May 12 22:50:53 +0800 2020,4503892254244050,4503877749088959,4503877749088959,0,回复@青荼蘼:你觉得冠姓权再重要，当事人觉得不重要。那你们还要强迫别人啊？一个人作出任何选择...,0,"{'id': 5638974069, 'idstr': '5638974069', 'cla...",4503892254244050,4503892254244050,NaN,NaN
2,Tue May 12 19:31:19 +0800 2020,4503842027558210,4503842027558210,4503842027558210,235,我笑了。男的可是宁愿杀妻也不要孩子随母姓，男的宁愿离婚不要孩子随母姓……呢。姓氏不重要哈[嘻...,0,"{'id': 6927664310, 'idstr': '6927664310', 'cla...",4503842027558210,4503842027558210,"{'danmaku_time': '52339', 'danmaku_from': '0'}",NaN
3,Tue May 12 22:50:15 +0800 2020,4503892094632524,4503877749088959,4503877749088959,0,回复@青荼蘼:就你这阴阳怪气的评论，普通人看到只会厌恶，这对女权的推广是极为不利的，戾气太重...,0,"{'id': 3882791691, 'idstr': '3882791691', 'cla...",4503892094632524,4503892094632524,NaN,NaN
4,Tue May 12 21:12:49 +0800 2020,4503867574885068,4503825493524912,4503825493524912,0,回复@我给老板画大饼:欧美婚都不怎么结了，法国非婚生子2007年就超过百分之六十了[摊手],0,"{'id': 2146690095, 'idstr': '2146690095', 'cla...",4503867574885068,4503867574885068,NaN,NaN


In [103]:
#有些有有些没，我们把这些有回复的提取出来，看看他们回复谁的评论
pd.DataFrame(list(pd.DataFrame(list(result[-result['reply_comment'].isnull()]['reply_comment']))['user']))['name']

0            一块废柄
1         我家狗子叫小猫
2         喷火的龙之传人
3         脾气暴躁的乐乐
4      总有一天我也会有猫的
          ...    
532        爱足球的小辛
533          鹧鸪孤鶱
534       红尘陌上097
535          鹧鸪孤鶱
536        爱足球的小辛
Name: name, Length: 537, dtype: object

In [106]:
#赋值回去
result.loc[-result['reply_comment'].isnull(),'replyname'] = list(pd.DataFrame(list(pd.DataFrame(list(result[-result['reply_comment'].isnull()]['reply_comment']))['user']))['name'])

In [107]:
#看看对不对
#username是发评论的用户
#replyname是他们回复的评论的用户
result[-result['reply_comment'].isnull()]

,comment_badge,created_at,disable_reply,extend_info,floor_number,id,idstr,mid,readtimetype,reply_comment,reply_original_text,rootid,rootidstr,status,text,user,username,replyname
2,NaN,Tue May 12 23:01:01 +0800 2020,0,NaN,0,4503894799684876,4503894799684876,4503894799684876,comment,{'created_at': 'Tue May 12 20:28:14 +0800 2020...,那没事了[允悲][允悲]可能是我没见到吧，我的,4503827548019096,4503827548019096,"{'visible': {'type': 0, 'list_id': 0}, 'create...",回复@一块废柄:那没事了[允悲][允悲]可能是我没见到吧，我的,"{'id': 5690080210, 'idstr': '5690080210', 'cla...",ZWWzwc,一块废柄
3,NaN,Tue May 12 22:57:36 +0800 2020,0,NaN,0,4503893939440479,4503893939440479,4503893939440479,comment,{'created_at': 'Tue May 12 22:50:53 +0800 2020...,谁强迫她了？恩和开始说的也只是陈述句“孩子随父姓”，现在整的我以为有拿枪逼着她给娃改姓呢。不...,4503877749088959,4503877749088959,"{'visible': {'type': 0, 'list_id': 0}, 'create...",回复@我家狗子叫小猫:谁强迫她了？恩和开始说的也只是陈述句“孩子随父姓”，现在整的我以为有拿...,"{'id': 2274183742, 'idstr': '2274183742', 'cla...",青荼蘼,我家狗子叫小猫
4,NaN,Tue May 12 22:57:26 +0800 2020,0,NaN,0,4503893901477855,4503893901477855,4503893901477855,comment,{'created_at': 'Tue May 12 19:31:19 +0800 2020...,张小美那青春又美丽的脸被我颜射了？我自己也彻底懵逼了。而接下来我要面对的是张小美的勃然大怒—...,4503842027558210,4503842027558210,"{'visible': {'type': 0, 'list_id': 0}, 'create...",回复@喷火的龙之传人:张小美那青春又美丽的脸被我颜射了？我自己也彻底懵逼了。而接下来我要面对...,"{'id': 1711692274, 'idstr': '1711692274', 'cla...",曲别流殇_898,喷火的龙之传人
5,NaN,Tue May 12 22:54:52 +0800 2020,0,NaN,0,4503893255845187,4503893255845187,4503893255845187,comment,{'created_at': 'Tue May 12 22:50:15 +0800 2020...,我又不结婚，冠姓权本来也不关我事儿？女权本来就是喜欢就来，不爽就走，又没组织没纪律。咋，还发...,4503877749088959,4503877749088959,"{'visible': {'type': 0, 'list_id': 0}, 'create...",回复@脾气暴躁的乐乐:我又不结婚，冠姓权本来也不关我事儿？女权本来就是喜欢就来，不爽就走，又...,"{'id': 2274183742, 'idstr': '2274183742', 'cla...",青荼蘼,脾气暴躁的乐乐
6,NaN,Tue May 12 22:54:01 +0800 2020,0,NaN,0,4503893037568336,4503893037568336,4503893037568336,comment,{'created_at': 'Tue May 12 21:12:49 +0800 2020...,可惜他们看不到这些,4503825493524912,4503825493524912,"{'visible': {'type': 0, 'list_id': 0}, 'create...",回复@总有一天我也会有猫的:可惜他们看不到这些,"{'id': 7183137389, 'idstr': '7183137389', 'cla...",孝道将人奴化,总有一天我也会有猫的
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22,NaN,Tue May 12 18:22:49 +0800 2020,0,NaN,0,4503824792754234,4503824792754234,4503824792754234,comment,{'created_at': 'Tue May 12 18:18:47 +0800 2020...,为什么打在兄弟上，没有姐妹， 为什么是公屏，不是母屏，恶臭蝻权就是你吧[doge][doge...,4503823777926509,4503823777926509,"{'visible': {'type': 0, 'list_id': 0}, 'create...",回复@爱足球的小辛:为什么打在兄弟上，没有姐妹， 为什么是公屏，不是母屏，恶臭蝻权就是你吧[...,"{'id': 2117684405, 'idstr': '2117684405', 'cla...",Digital曦,爱足球的小辛
23,NaN,Tue May 12 18:22:48 +0800 2020,0,NaN,0,4503824788198275,4503824788198275,4503824788198275,comment,{'created_at': 'Tue May 12 18:18:19 +0800 2020...,NaN,4503823655872375,4503823655872375,"{'visible': {'type': 0, 'list_id': 0}, 'create...","微博上的""女权主义"": 你这个女人怎么跟我的观念不一样，我要为女性争取权利，我要先把你这头母...","{'id': 7189228727, 'idstr': '7189228727', 'cla...",炒饭大师199402,鹧鸪孤鶱
35,[{'pic_url': 'https://h5.sinaimg.cn/upload/100...,Tue May 12 18:20:53 +0800 2020,0,NaN,0,4503824305431543,4503824305431543,4503824305431543,comment,{'created_at': 'Tue May 12 18:19:11 +0800 2020...,卑鄙是卑鄙者的通行证,4503823877728490,4503823877728490,"{'visible': {'type': 0, 'list_id': 0}, 'create...",回复@红尘陌上097:卑鄙是卑鄙者的通行证,"{'id': 2426432731, 'idstr': '2426432731', 'cla...",Artemis婷婷_貂爷家皮草,红尘陌上097
37,NaN,Tue May 12 18:20:49 +0800 2020,0,NaN,0,4503824289659100,4503824289659100,4503824289659100,comment,{'created_at': 'Tue May 12 18:18:19 +0800 2020...,对啊，现在网络喷子质量堪忧啊 哈哈哈哈哈哈 搞不懂人家生孩子随谁姓关他们啥事。还是吃的太饱...,4503823655872375,4503823655872375,"{'visible': {'type': 0, 'list_id': 0}, 'create...",回复@鹧鸪孤鶱:对啊，现在网络喷子质量堪忧啊 哈哈哈哈哈哈 搞不懂人家生孩子随谁姓关他们啥...,"{'id': 6130736773, 'idstr': '6130736773', 'cla...",肉肉肉肉啊08256,鹧鸪孤鶱


OK下一步来可视化

可视化要的数据是两部分:
>1.一部分是从这个微博的发布者到评论者的关系，以评论数量为关系的权重  
2.第二部分是评论者与评论者之间的关系，以互相回复的数量为关系的权重

In [171]:
#第一部分
datatoplot1 = result[result['replyname'].isnull()].groupby(['username'])['mid'].count().reset_index()
datatoplot1.columns = ['D','count']
datatoplot1['O'] = '观察者网'
datatoplot1[['O','D','count']].head(5)

,O,D,count
0,观察者网,1701__,2
1,观察者网,April1386,9
2,观察者网,Artemis婷婷_貂爷家皮草,2
3,观察者网,Digital曦,5
4,观察者网,GeraIt_Of_Rivia,3


In [172]:
#第二部分
datatoplot2 = result.groupby(['username','replyname'])['mid'].count().reset_index()
datatoplot2.columns = ['O','D','count']
datatoplot2.head(5)

,O,D,count
0,April1386,Digital曦,2
1,April1386,千玺的朱砂痣,1
2,April1386,想个名字真麻烦23333,1
3,April1386,我给老板画大饼,2
4,April1386,樱语嫣然1996,1


In [173]:
#合体！
link = pd.concat([datatoplot1,datatoplot2])
link.head(5)

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  


,D,O,count
0,1701__,观察者网,2
1,April1386,观察者网,9
2,Artemis婷婷_貂爷家皮草,观察者网,2
3,Digital曦,观察者网,5
4,GeraIt_Of_Rivia,观察者网,3


In [174]:
#然后就是，算一下每个人的发言频次！
node = link.groupby(['O'])['count'].sum().reset_index().sort_values(by = 'count',ascending = False)
node.head(5)

,O,count
73,观察者网,292
57,炒饭大师199402,78
80,青荼蘼,27
34,太座猛于虎,27
27,十四个夏,21


# 可视化

这回我们用到的是pyecharts里面的[Graph关系图](https://pyecharts.org/#/zh-cn/basic_charts?id=graph%ef%bc%9a%e5%85%b3%e7%b3%bb%e5%9b%be)！


查阅echarts的配置项手册可以看到需要的数据格式[graph的配置项手册](https://echarts.apache.org/zh/option.html#series-graph.data)

In [175]:
#把node数据整理一下
node_json = list(node.apply(lambda r:{'name':r['O'],'value':r['count']},axis = 1))

In [176]:
#把link数据整理一下
link_json = list(link.apply(lambda r:{'source':r['O'],'target':r['D']},axis = 1))

In [177]:
#pyecharts，走起！
from pyecharts import options as opts
from pyecharts.charts import Graph
c = (
    Graph()
    .add("weibo", 
         nodes = node_json, 
         links = link_json
        )
    .set_global_opts(
        title_opts=opts.TitleOpts(title='微博评论关系图')
    )
)
c.render('微博评论关系图.html')

'C:\\pythonscript\\2020\\pygeo-tutorial\\微博评论关系图.html'

效果如下
<img src="https://gitee.com/ni1o1/pygeo-tutorial/raw/master/resource/weibo3.png"  style="width:600px">

In [178]:
#把node数据整理一下,加上点的大小
node_json = list(node.apply(lambda r:{'name':r['O'],'value':r['count'],'symbolSize':r['count']**0.5},axis = 1))
#把link数据整理一下,加上边的粗细
link_json = list(link.apply(lambda r:{'source':r['O'],'target':r['D'],'lineStyle':{'width':0.5*r['count']**0.5}},axis = 1))

In [179]:
#pyecharts，走起！
from pyecharts import options as opts
from pyecharts.charts import Graph
c = (
    Graph()
    .add("weibo", 
         nodes = node_json, 
         links = link_json
        )
    .set_global_opts(
        title_opts=opts.TitleOpts(title='微博评论关系图')
    )
)
c.render('微博评论关系图.html')

'C:\\pythonscript\\2020\\pygeo-tutorial\\微博评论关系图.html'

效果如下
<img src="https://gitee.com/ni1o1/pygeo-tutorial/raw/master/resource/weibo4.png"  style="width:600px">